#### First load the resnet output data and build the model 

In [3]:
import numpy as np

y = np.load('ytraindata.npy')

x = np.load('x_resnet152v2.npy')


In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [3]:
model = Sequential([
    Rescaling(1./255, input_shape=(12, 16, 2048)),
    Flatten(),
    Dense(600, activation='relu'),
    Dropout(0.75),
    Dense(600, activation='relu'),
    Dropout(0.75),
    Dense(600, activation='relu'),
    Dropout(0.75),
    Dense(6, activation='softmax')
])

In [4]:
from sklearn.model_selection import train_test_split

xTrain, xVal, yTrain, yVal = train_test_split(x,y,random_state=1)

#### Set the learning rate and momentum accordingly to speedup training or to slow down and fine tune

In [5]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(learning_rate=0.008, momentum=0.9), metrics=['accuracy'])

In [16]:
model.fit(x = xTrain, y = yTrain,
          epochs=20,
          validation_data=(xVal,yVal))

Epoch 1/20
30/30 [==============================] - 16s 547ms/step - loss: 0.4477 - accuracy: 0.8448 - val_loss: 0.6387 - val_accuracy: 0.7627
Epoch 2/20
30/30 [==============================] - 16s 546ms/step - loss: 0.4011 - accuracy: 0.8659 - val_loss: 0.5418 - val_accuracy: 0.8101
Epoch 3/20
30/30 [==============================] - 16s 546ms/step - loss: 0.3440 - accuracy: 0.8891 - val_loss: 0.5854 - val_accuracy: 0.7911
Epoch 4/20
30/30 [==============================] - 16s 548ms/step - loss: 0.2892 - accuracy: 0.9007 - val_loss: 0.5820 - val_accuracy: 0.8070
Epoch 5/20
30/30 [==============================] - 16s 547ms/step - loss: 0.2912 - accuracy: 0.9039 - val_loss: 0.5449 - val_accuracy: 0.8133
Epoch 6/20
30/30 [==============================] - 17s 553ms/step - loss: 0.2556 - accuracy: 0.9124 - val_loss: 0.5598 - val_accuracy: 0.8386
Epoch 7/20
30/30 [==============================] - 16s 550ms/step - loss: 0.2249 - accuracy: 0.9293 - val_loss: 0.5463 - val_accuracy: 0.8196

KeyboardInterrupt: 

#### Once you've tuned the model above, train the model on all the data below, set epochs to the number that resulted in the best accuracy + a little bit, to account for the larger dataset

In [6]:
model.fit(x = x, y = y,
          epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
40/40 [==============================] - 22s 538ms/step - loss: 0.3721 - accuracy: 0.8725


### Now save the model weights to disk so that you don't have to train the model again, also at this point you can shut down the kernel if you're running low on RAM, since the model is saved to disk we don't need to store the original arrays and etc in memory 

In [7]:
model.save_weights('temp_saved_weights')

### Now reload the model, the architecture here should be identical to what you have above so if you've changed any of the layers make sure you copy/paste them in

In [1]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
model = Sequential([
    Rescaling(1./255, input_shape=(12, 16, 2048)),
    Flatten(),
    Dense(600, activation='relu'),
    Dropout(0.75),
    Dense(600, activation='relu'),
    Dropout(0.75),
    Dense(600, activation='relu'),
    Dropout(0.75),
    Dense(6, activation='softmax')
])


#### Now load the saved weights and the following code will generate the kaggle predictions

In [2]:
model.load_weights('temp_saved_weights')

In [3]:
xkaggle = np.load('x_test_resnet152V2.npy')

In [4]:
testpreds = np.argmax(model.predict(xkaggle), axis=-1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [5]:
import pandas as pd

kagglesub = pd.DataFrame(testpreds+1)
kagglesub.columns = ['Prediction']

kagglesub['Id'] = np.arange(1264,2528,1)

kagglesub.set_index('Id',inplace=True)
kagglesub.to_csv('kagglesubmission.txt')